# 1. Audit the uncleaned data of osm_samples.db.

In [45]:
#Connect to osm_samples.db using sqlite3.
#First, query what are the keys of tags.
import sqlite3

db = sqlite3.connect("C:\\sqlite_windows\\osm_samples.db")
c = db.cursor()

In [46]:
#query to know the types of tags.
uncleaned_tags_query = "select distinct tags.key as key \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        order by key;"
c.execute(uncleaned_tags_query)
uncleaned_tags = c.fetchall()
for row in uncleaned_tags:
    print(row)

('Bergen_County_database_ref',)
('FIXME',)
('Fax',)
('HFCS',)
('NHD:AUXCODE',)
('NHD:BldgAddr_1',)
('NHD:BldgName_1',)
('NHD:BldgNum',)
('NHD:CampusCo_1',)
('NHD:City_1',)
('NHD:ComID',)
('NHD:FCode',)
('NHD:FDate',)
('NHD:FTYPE',)
('NHD:FType',)
('NHD:Lat_1',)
('NHD:Long_1',)
('NHD:RESOLUTION',)
('NHD:ReachCode',)
('NHD:State_1',)
('NHD:Zip_1',)
('NHD:way_id',)
('NHS',)
('NJDOT_SRI',)
('NOTE',)
('NYDEC_Lands:CATEGORY',)
('NYDEC_Lands:COUNTY',)
('NYDEC_Lands:FACILITY',)
('abandoned',)
('access',)
('admin_level',)
('aerialway',)
('aeroway',)
('agricultural',)
('alt_name',)
('amenity',)
('area',)
('artist_name',)
('artwork_type',)
('atc',)
('atm',)
('attraction',)
('attribution',)
('auxcode',)
('backrest',)
('backward',)
('bar',)
('barrier',)
('basin',)
('beauty',)
('bell',)
('bench',)
('bicycle',)
('bicycle_parking',)
('bidirectional',)
('bin',)
('bitcoin',)
('black_owned',)
('board_type',)
('boat',)
('border_type',)
('both_ways',)
('boundary',)
('boundary_type',)
('brand',)
('bridge',)

In [47]:
#query what are the min and max value of latitude and longitude.
lat_lon_query = "select min(lat), max(lat), min(lon), max(lon) \
        from uncleaned_nodes;"
c.execute(lat_lon_query)
lat_lon = c.fetchall()
for row in lat_lon:
    print(row)

(40.345, 41.0969983, -74.5009765, -73.2260147)


- It looked like the latitudes and longitudes are in right range compared to this [wiki page](https://en.wikipedia.org/wiki/New_York_(state)).

In [48]:
#query about postcodes.
uncleaned_postcode_query = "select tags.value, count(*) as count \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'postcode' \
        group by tags.value \
        order by count desc;"
c.execute(uncleaned_postcode_query)
uncleaned_postcodes = c.fetchall()
for row in uncleaned_postcodes:
    print(row)

('10314', 769)
('11234', 678)
('10312', 594)
('10306', 534)
('11236', 510)
('11706', 507)
('11385', 502)
('11746', 477)
('11743', 468)
('11757', 438)
('11229', 411)
('11203', 398)
('10469', 388)
('11207', 369)
('11434', 366)
('10305', 362)
('11208', 360)
('11223', 360)
('11704', 355)
('11357', 346)
('11204', 345)
('11731', 342)
('10466', 328)
('11221', 325)
('11214', 324)
('11215', 323)
('10465', 319)
('11413', 319)
('11210', 315)
('11725', 315)
('11219', 314)
('10309', 311)
('11358', 307)
('11220', 306)
('11420', 306)
('10304', 302)
('11209', 300)
('11412', 300)
('11377', 298)
('11379', 296)
('11717', 292)
('10308', 290)
('11235', 284)
('10301', 283)
('10461', 283)
('11230', 281)
('11364', 280)
('11795', 280)
('11228', 279)
('11233', 274)
('11729', 271)
('11378', 267)
('11368', 262)
('11375', 261)
('11419', 260)
('11365', 250)
('11373', 250)
('11432', 250)
('11768', 248)
('11218', 246)
('11355', 241)
('11212', 230)
('11361', 230)
('11422', 227)
('11433', 223)
('11435', 221)
('11367', 

- Most of postcodes start from 10 or 11. 
- And some of them start from 08 or 07 or 06 and they mean New Jersey or Connecticut. I used [these](http://www.geonames.org/postal-codes/postal-codes-us.html) [sites](https://en.wikipedia.org/wiki/New_York_metropolitan_area). Therefore I can infer that this New York OSM data has information about not just New York state, but New York Metropolitan area.
- There are 3 different types of postal codes. First, 5 digit code. Second, 5 digit code added by 4 digits connected by '-'. Third, 5 digit code started by 'NY '.I think that I can programmatically unify them. I'm going to use 5 digit code and zip+4 code both because zip+4 code is better at mailing([reference](https://smartystreets.com/articles/zip-4-code))
- There are 2 or 3 or 6 digit code and I think that it was input error. When cleaning the data before sending into DB, I'll ignore them.

In [49]:
#query about emails.
uncleaned_email_query = "select tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'email' \
        order by tags.value;"
c.execute(uncleaned_email_query)
uncleaned_emails = c.fetchall()
for row in uncleaned_emails:
    print(row)

('bathenhaven@yahoo.com',)
('info@jadisnyc.com',)
('info@marrakechhotelnyc.com',)
('james@gsalehouse.com',)
('madison@cluizel.us',)
('support@laserengravenj.com',)


- There are just 6 emails in this sample and there isn't any wrong in them.

In [50]:
#query about facebooks.
uncleaned_fb_query = "select tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'facebook' \
        order by tags.value;"
c.execute(uncleaned_fb_query)
uncleaned_fb = c.fetchall()
for row in uncleaned_fb:
    print(row)

('https://www.facebook.com/pages/MacDougal-St-Ale-House-Pla-Street/116550071739296',)


- There are just 1 facebook page in this sample and there isn't any wrong in it.

In [51]:
#query about websites.
uncleaned_site_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'website' \
        order by tags.value;"
c.execute(uncleaned_site_query)
uncleaned_sites = c.fetchall()
for row in uncleaned_sites:
    print(row)

('baileyarboretum.org',)
('bigtreebottles.com',)
('bk.com',)
('brookwoodatbayshore.com',)
('hsbc.com',)
('http://527madisonave.com/',)
('http://825thirdave.com/',)
('http://auttharosthainewyork.com',)
('http://bathenhaven.com/',)
('http://brooklynsandbox.com',)
('http://brooklynsocialbar.com/',)
('http://celticcorner.us/',)
('http://charno4.com/',)
('http://citibikenyc.com',)
('http://davoodzadehrugs.com',)
('http://doriangraynyc.com',)
('http://drambar.com/',)
('http://ezcesspoollongisland.com',)
('http://fkh.clarkschools.org/',)
('http://gomusicworks.com',)
('http://greenwichlocksmiths.com',)
('http://gsalehouse.com/',)
('http://jadisnyc.com/',)
('http://jerichoanimalhospital.com',)
('http://keyportlibrary.org/',)
('http://kfsmortgage.com',)
('http://landmarklivingston.com/',)
('http://lennysnyc.com/stores#lenny-s-55th-street',)
('http://lfstores.com/',)
('http://livinglotsnyc.org/lot/58276/',)
('http://location.ctownsupermarkets.com/ny/jamaica/41-003/',)
('http://lucasstamford.com/'

- There are three different forms of website addresses. First, URL started from 'http' or 'https' scheme. Second, domain name started from 'www'. Third, domain name without 'www'. I think that it is better to make all website addresses which don't start with 'http' or 'https' start with 'http://' for the consistency. And the other reason is that even if there are some sites which don't use 'www' subdomain, there isn't any websites which can be accessed without using http protocol. Referenced [these](https://en.wikipedia.org/wiki/URL) [sites](https://stackoverflow.com/questions/8951423/html-links-without-http-protocol).

In [52]:
#query about phones.
uncleaned_phone_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'phone' \
        order by tags.value;"
c.execute(uncleaned_phone_query)
uncleaned_phones = c.fetchall()
for row in uncleaned_phones:
    print(row)

('(201) 444-8111',)
('(201) 612-0756',)
('(212) 260-8015',)
('(212) 340-0863',)
('(212) 675-0550',)
('(212) 982-0362',)
('(212)228-5483',)
('(347) 799-2640',)
('(516) 431-5431',)
('(631) 499-8580',)
('(718) 229-7600',)
('(718) 230-3881',)
('(718) 321-6893',)
('(718) 369-0033',)
('(718) 381-5136',)
('(718) 488-7005',)
('(718) 623-9065',)
('(718) 677-5811',)
('(718) 821-8085',)
('(718) 855-9600',)
('(732) 264-7983',)
('(855) 417-1284',)
('(973) 256-7742',)
('(973) 310-3650',)
('(973) 564-9092',)
('+1 (201) 398-0944',)
('+1 (516) 801-6544',)
('+1 (718) 250-0200',)
('+1 (718) 387-9617',)
('+1 203 863 3000',)
('+1 205 9007007',)
('+1 212 2222954',)
('+1 212 242 6200',)
('+1 212 254 0006',)
('+1 212 353-3780',)
('+1 212 502 4609',)
('+1 212 663 1580',)
('+1 212 663 6004',)
('+1 212 665 9800',)
('+1 212 675-0342',)
('+1 212 690 4000',)
('+1 212 749 1111',)
('+1 212 799-9010',)
('+1 212 874 4000',)
('+1 212 9660290',)
('+1 212 989 8737',)
('+1 212-254-1675',)
('+1 516 783 86 60',)
('+1 646 415

- There are lots of different types of phone numbers in the result. I'm going to make them have ['(NPA) NXX-XXXX' format](https://en.wikipedia.org/wiki/National_conventions_for_writing_telephone_numbers), where NPA is the numbering plan area code and NXX-XXXX is the subscriber number.

In [53]:
#query about opening-hours.
uncleaned_oh_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'opening_hours' \
        order by tags.value;"
c.execute(uncleaned_oh_query)
uncleaned_ohs = c.fetchall()
for row in uncleaned_ohs:
    print(row)

('11:00 AM to Close',)
('24/7',)
('5pm - 9pm (weekdays) , 2pm - 8pm ( weekend)',)
('772',)
('Mo-Fr 05:00-21:00, Su 06:30-21:00, Sa 06:30-21:00, PH 07:00-19:00',)
('Mo-Fr 05:30-21:30; Sa-Su 07:00-21:30',)
('Mo-Fr 05:30-22:00; Sa-Su 06:00-22:00',)
('Mo-Fr 07:00-19:00; Sa-Su 08:00-16:00',)
('Mo-Fr 07:00-20:00; Sa-Su 08:00-20:00',)
('Mo-Fr 07:00-21:00; Sa 08:00-20:00; Su 09:00-19:00',)
('Mo-Fr 07:30-18:00; Sa,Su 08:00-16:00',)
('Mo-Fr 07:30-19:00; Sa 07:30-16:00; Su 11:00-15:00',)
('Mo-Fr 08:00-18:00',)
('Mo-Fr 08:30-17:00; Sa 10:00-14:00',)
('Mo-Fr 08:30-18:00; Sa 09:00-15:00',)
('Mo-Fr 09:00-17:00',)
('Mo-Fr 09:00-20:00; Sa-Su 10:00-18:00',)
('Mo-Fr 10:00-19:30; Sa 10:00-18:00; Su 10:00-17:00',)
('Mo-Fr 10:00-20:00; Sa 10:00-18:00; Su 11:00-17:00',)
('Mo-Fr 11:00-22:00; Sa-Su 12:00-22:00',)
('Mo-Fr 12:00-16:30, 17:00-23:00; Sa-Su 12:00-23:30',)
('Mo-Fr 13:00-01:00; Sa-Su 12:00-01:00',)
('Mo-Fr 14:00-04:00',)
('Mo-Sa 07:00-19:00',)
('Mo-Sa 07:00-22:00',)
('Mo-Sa 08:00-04:00; Su 12:00-04:0

- I'll skip cleaning it. The result is too complicated for me.

In [54]:
#query about faxs.
uncleaned_fax_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'fax' \
        order by tags.value;"
c.execute(uncleaned_fax_query)
uncleaned_faxes = c.fetchall()
for row in uncleaned_faxes:
    print(row)

('+1 212 242 1278',)
('+1 212 678 6842',)
('+1 732 5432409',)
('+1 973 761 9193',)
('+1-212-268-0847',)
('718-268-7382',)
('718-361-2734',)


In [55]:
#query about Faxs.
uncleaned_Fax_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'Fax' \
        order by tags.value;"
c.execute(uncleaned_Fax_query)
uncleaned_Faxes = c.fetchall()
for row in uncleaned_Faxes:
    print(row)

('732-548-8887',)


- 'Fax' and 'fax' are same but divided by capital or small letter and the results are like phone number's. I'm going to alter the key 'Fax' to 'fax' and make the numbers have the same format as phone number's.

In [56]:
#query about maxspeed.
uncleaned_maxspeed_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'maxspeed' \
        order by tags.value;"
c.execute(uncleaned_maxspeed_query)
uncleaned_maxspeeds = c.fetchall()
for row in uncleaned_maxspeeds:
    print(row)

('10',)
('10 mph',)
('110 mph',)
('125 mph',)
('15',)
('15 mph',)
('160',)
('177',)
('20 mph',)
('201',)
('217',)
('25 mph',)
('30',)
('30 mph',)
('35',)
('35 mph',)
('40 mph',)
('45 mph',)
('48',)
('50',)
('50 mph',)
('55 mph',)
('60',)
('60 mph',)
('64',)
('65 mph',)
('70 mph',)
('72',)
('75 mph',)
('80',)
('90 mph',)
('Amtrak ETT 2010-01-18',)
('Conrail ETT 2013-07-01',)
('Conrail ETT 2013-10-17',)
('signals',)


- Some of them are just number and others are number added by 'mph'. I'm going to remove unrelated datas and add 'mph' to all numbers. New York [speed limit](https://en.wikipedia.org/wiki/Speed_limits_in_the_United_States_by_jurisdiction#New_York), 

In [57]:
#query about street names.
uncleaned_stname_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'street' \
        order by tags.value;"
c.execute(uncleaned_stname_query)
uncleaned_stnames = c.fetchall()
for row in uncleaned_stnames:
    print(row)

('100th Avenue',)
('100th Drive',)
('100th Street',)
('101st Avenue',)
('101st Street',)
('102nd Avenue',)
('102nd Road',)
('102nd Street',)
('103rd Avenue',)
('103rd Road',)
('103rd Street',)
('104th Avenue',)
('104th Street',)
('105th Avenue',)
('105th Street',)
('106th Avenue',)
('106th Street',)
('107th Avenue',)
('107th Street',)
('108th Avenue',)
('108th Drive',)
('108th Street',)
('109th Avenue',)
('109th Drive',)
('109th Road',)
('109th Street',)
('10th Avenue',)
('10th Street',)
('110th Avenue',)
('110th Road',)
('110th Street',)
('111th Avenue',)
('111th Road',)
('111th Street',)
('112th Avenue',)
('112th Place',)
('112th Road',)
('112th Street',)
('113th Avenue',)
('113th Drive',)
('113th Road',)
('113th Street',)
('114th Avenue',)
('114th Drive',)
('114th Place',)
('114th Road',)
('114th Street',)
('114th Terrace',)
('115th Avenue',)
('115th Court',)
('115th Drive',)
('115th Road',)
('115th Street',)
('116th Avenue',)
('116th Drive',)
('116th Road',)
('116th Street',)
('117

('70th Street',)
('71st Ave',)
('71st Avenue',)
('71st Crescent',)
('71st Drive',)
('71st Place',)
('71st Road',)
('71st Street',)
('72nd Avenue',)
('72nd Crescent',)
('72nd Drive',)
('72nd Place',)
('72nd Road',)
('72nd Street',)
('73rd Avenue',)
('73rd Place',)
('73rd Road',)
('73rd Street',)
('73rd Terrace',)
('74th Avenue',)
('74th Place',)
('74th Street',)
('75th Avenue',)
('75th Place',)
('75th Road',)
('75th Street',)
('76th Avenue',)
('76th Drive',)
('76th Road',)
('76th Street',)
('77th Avenue',)
('77th Crescent',)
('77th Place',)
('77th Road',)
('77th Street',)
('78th Avenue',)
('78th Drive',)
('78th Road',)
('78th Street',)
('79th Avenue',)
('79th Place',)
('79th Street',)
('7th Avenue',)
('7th Avenue South',)
('7th Avenue W',)
('7th St',)
('7th Street',)
('80TH Street',)
('80th Avenue',)
('80th Drive',)
('80th Road',)
('80th Street',)
('81st Avenue',)
('81st Street',)
('82nd Avenue',)
('82nd Drive',)
('82nd Place',)
('82nd Road',)
('82nd Street',)
('83rd Avenue',)
('83rd Pl

('Beach 42nd Street',)
('Beach 43rd Street',)
('Beach 44th Street',)
('Beach 45th Street',)
('Beach 46th Street',)
('Beach 47th Street',)
('Beach 48th Street',)
('Beach 54th Street',)
('Beach 58th Street',)
('Beach 59th Street',)
('Beach 5th Street',)
('Beach 61st Street',)
('Beach 63rd Street',)
('Beach 64th Street',)
('Beach 65th Street',)
('Beach 66th Street',)
('Beach 67th Street',)
('Beach 68th Street',)
('Beach 69th Street',)
('Beach 70th Street',)
('Beach 72nd Street',)
('Beach 73rd Street',)
('Beach 74th Street',)
('Beach 75th Street',)
('Beach 76th Street',)
('Beach 7th Street',)
('Beach 84th Street',)
('Beach 86th Street',)
('Beach 87th Street',)
('Beach 88th Street',)
('Beach 89th Street',)
('Beach 90th Street',)
('Beach 91st Street',)
('Beach 92nd Street',)
('Beach 93rd Street',)
('Beach 94th Street',)
('Beach 95th Street',)
('Beach 96th Street',)
('Beach 98th Street',)
('Beach 9th Street',)
('Beach Avenue',)
('Beach Channel Drive',)
('Beach Plum Drive',)
('Beach Prom',)
('

('Catalpa Avenue',)
('Catalpa Drive',)
('Catalpa Street',)
('Catamaran Way',)
('Cathedral Parkway',)
('Catherine Street',)
('Catlin Avenue',)
('Caton Avenue',)
('Caton Place',)
('Caton Street',)
('Catskill Avenue',)
('Cauldwell Avenue',)
('Cayuga Avenue',)
('Cayuga Place',)
('Cebra Avenue',)
('Cecil Avenue',)
('Cedar Avenue',)
('Cedar Court',)
('Cedar Crest Drive',)
('Cedar Drive',)
('Cedar Grove Court',)
('Cedar Grove Lane',)
('Cedar Hill Road',)
('Cedar Lane',)
('Cedar Place',)
('Cedar Point Drive',)
('Cedar Ridge Lane',)
('Cedar Road',)
('Cedar Street',)
('Cedar Street N',)
('Cedar Valley Lane',)
('Cedarcroft Road',)
('Cedartree Lane',)
('Cedarview Avenue',)
('Cedrus Avenue',)
('Celebration Lane',)
('Celeste Court',)
('Celia Street',)
('Centennial Avenue',)
('Center Avenue',)
('Center Bay Drive',)
('Center Chicot Avenue',)
('Center Drive',)
('Center Drive Malba',)
('Center Lane',)
('Center Place',)
('Center Street',)
('Centerport Road',)
('Centershore Road',)
('Centerwood Street',)


('East Hampton Boulevard',)
('East Houston Street',)
('East Kingsbridge Road',)
('East Loop Road',)
('East Macon Avenue',)
('East Main Street',)
('East Mosholu Parkway North',)
('East Mt Eden Avenue',)
('East New York Avenue',)
('East Raleigh Avenue',)
('East Ridgewood Avenue',)
('East Stroud Avenue',)
('East Tremont Avenue',)
('East Williston Avenue',)
('Eastbrook Court',)
('Eastburn Avenue',)
('Eastchester Place',)
('Eastchester Road',)
('Eastern Avenue',)
('Eastern Parkway',)
('Eastern Parkway Extension',)
('Easton Avenue',)
('Easton Street',)
('Eastwood Avenue',)
('Eastwood Drive',)
('Eastwoods Drive',)
('Eatons Neck Road',)
('Ebb Court',)
('Ebbitts Street',)
('Ebbtide Lane',)
('Ebey Lane',)
('Ebony Court',)
('Echo Lane',)
('Echo Place',)
('Ecker Avenue',)
('Eckford Avenue',)
('Eckford Street',)
('Edcris Lane',)
('Eddie Avenue',)
('Eddy Street',)
('Edel Avenue',)
('Eden Road',)
('Edenwald Avenue',)
('Edgecombe Avenue',)
('Edgegrove Avenue',)
('Edgehill Avenue',)
('Edgemere Avenue',

...

('Latourette Lane',)
('Latourette Street',)
('Laurel Avenue',)
('Laurel Drive',)
('Laurel Hill Boulevard',)
('Laurel Hill Road',)
('Laurel Road',)
('Laurelton Parkway',)
('Laurelton Parkway Service Road South',)
('Lauren Avenue',)
('Laurie Court',)
('Laurie Road',)
('Lava Street',)
('Lawn Avenue',)
('Lawrence Avenue',)
('Lawrence Lane',)
('Lawrence Road',)
('Lawrence Street',)
('Lawton Avenue',)
('Layton Avenue',)
('Le Britton Street',)
('Leader Avenue',)
('Leaf Court',)
('Leavitt Street',)
('Lebanon Street',)
('Lee Avenue',)
('Lee Lane',)
('Lee Street',)
('Leeds Lane',)
('Leeds Road',)
('Leeland Drive',)
('Lefferts Avenue',)
('Lefferts Boulevard',)
('Lefferts Place',)
('Legends Cir',)
('Leggett Avenue',)
('Leghorn Court',)
('Legion Drive',)
('Legion Place',)
('Legion Street',)
('Leibrock Avenue',)
('Leith Place',)
('Leland Avenue',)
('Lemoine Avenue',)
('Lenevar Avenue',)
('Lenhart Street',)
('Lenisue Court',)
('Lenox Avenue',)
('Lenox Road',)
('Leo Lane',)
('Leo Street',)
('Leonard A

('Oakley Drive',)
('Oakley Place',)
('Oakley Street',)
('Oakmere Drive',)
('Oakrest Lane',)
('Oaktree Lane',)
('Oakville Street',)
('Oakwood Avenue',)
('Oakwood Drive',)
('Oakwood Road',)
('Oban Street',)
('Oberlin Street',)
('Occident Avenue',)
('Ocean Avenue',)
('Ocean Court',)
('Ocean Crest Boulevard',)
('Ocean Parkway',)
('Ocean Road',)
('Ocean Street',)
('Ocean Terrace',)
('Ocean View Avenue',)
('Oceana Terrace',)
('Oceania Street',)
('Oceanic Avenue',)
('Oceanside Avenue',)
('Odell Street',)
('Oder Avenue',)
('Offaly Street',)
('Ogden Avenue',)
('Ogden Road',)
('Ohio Avenue',)
('Ohio Street',)
('Ohm Avenue',)
('Olcott Street',)
('Old Mill Road',)
('Old Saw Mill River Road',)
('Old Town Road',)
('Oldfield Avenue',)
('Oldfield Street',)
('Olean Street',)
('Olga Lane',)
('Olinville Avenue',)
('Olive Street',)
('Olive Walk',)
('Oliver Place',)
('Olivia Court',)
('Olivia Street',)
('Olmstead Avenue',)
('Olympia Boulevard',)
('Olympia Place',)
('Onderdonk Avenue',)
('Oneida Avenue',)
(

('Radnor Road',)
('Railroad Avenue',)
('Railroad Street',)
('Raily Court',)
('Rainbow Avenue',)
('Rainbow Lane',)
('Raleigh Lane',)
('Raleigh Place',)
('Raleigh Street',)
('Ralph Avenue',)
('Ramapo Avenue',)
('Ramita Lane',)
('Ramona Avenue',)
('Ramondo Lane',)
('Ramsey Road',)
('Rancher Place',)
('Randall Avenue',)
('Randolph Drive',)
('Randolph Place',)
('Randolph Street',)
('Ranick Road',)
('Rankin Street',)
('Ransom Street',)
('Rapelye Street',)
('Raphael Street',)
('Raritan Avenue',)
('Raritan Road',)
('Raspberry Court',)
('Rathbun Avenue',)
('Raven Drive',)
('Ravenhurst Avenue',)
('Ravenwood Drive',)
('Rawlins Avenue',)
('Ray Street',)
('Raybor Road',)
('Raymond Place',)
('Raynor Street',)
('Reade Avenue',)
('Reade Street',)
('Reading Avenue',)
('Redbrook Court',)
('Redding Street',)
('Redgrave Avenue',)
('Redington Street',)
('Redleaf Lane',)
('Redmond Avenue',)
('Redwood Loop',)
('Reeder Street',)
('Reeds Mill Lane',)
('Reeve Place',)
('Reeves Avenue',)
('Regal Drive',)
('Regal

('Sheraden Avenue',)
('Sherbrooke Road',)
('Sheridan Avenue',)
('Sheridan Court',)
('Sheridan Place',)
('Sheridan Square',)
('Sherlock Place',)
('Sherman Avenue',)
('Sherman Avenue South',)
('Sherman Street',)
('Sherry Court',)
('Sherwood Court',)
('Sherwood Drive',)
('Sheryl Cres',)
('Shiel Avenue',)
('Shiloh Avenue',)
('Shiloh Street',)
('Shinbone Lane',)
('Shirley Avenue',)
('Shirley Court',)
('Shirra Avenue',)
('Shoal Drive',)
('Shore Avenue',)
('Shore Boulevard',)
('Shore Court',)
('Shore Drive',)
('Shore Drive S',)
('Shore Drive W',)
('Shore Front Parkway',)
('Shore Lane',)
('Shore Parkway',)
('Shore Road',)
('Shore Walk',)
('Shoreham Drive E',)
('Shoreham Drive W',)
('Shoreland Drive',)
('Shorewood Lane',)
('Short Court',)
('Shorthill Road',)
('Shotwell Avenue',)
('Sidney Court',)
('Sidney Place',)
('Siegfried Place',)
('Signs Road',)
('Sigourney Street',)
('Silas Court',)
('Silo Road',)
('Silver Lake Road',)
('Silver Road',)
('Silver Street',)
('Silverpine Drive',)
('Simmons La

('Twombly Avenue',)
('Tynan Street',)
('Tyndall Avenue',)
('Tyram Lane',)
('Tysen Street',)
('Tysens Lane',)
('US 46',)
('Udalia Court',)
('Udalia Road',)
('Udall Road',)
('Ulster Avenue',)
('Ulster Court',)
('Uncas Avenue',)
('Undercliff Avenue',)
('Underhill Avenue',)
('Undset Court',)
('Union Avenue',)
('Union Boulevard',)
('Union Court',)
('Union Hall Street',)
('Union Street',)
('Union Turnpike',)
('Union street',)
('Unionport Road',)
('United Nations Plaza',)
('University Avenue',)
('University Place',)
('Unqua Place',)
('Upland Drive',)
('Upper Lane',)
('Ursina Road',)
('Uss Iowa Circle',)
('Utica Avenue',)
('Utica Street',)
('Utica Walk',)
('Utopia Parkway',)
('Utter Avenue',)
('Valdemar Avenue',)
('Valentine Avenue',)
('Valentine Court',)
('Valentine Place',)
('Valesite Court',)
('Valhalla Drive',)
('Valley Avenue',)
('Valley Drive',)
('Valley Forge Drive',)
('Valley Road',)
('Valley Street',)
('Valleyview Drive',)
('Valleywood Road',)
('Van Allen Avenue',)
('Van Brunt Street'

In [58]:
db.close()

- There are really lots of formats of street names. There's no shortened version of street that I'm going to leave them as it is.

### - Finally, I decided to solve the problems in maxspeed, website, postcode, phone, fax, and Fax fields.

# 2. Audit the cleaned and uncleaned data of osm_small_samples.db.

In [59]:
#Connect to osm_small_samples.db using sqlite3.
import sqlite3

small_db = sqlite3.connect("C:\\sqlite_windows\\osm_small_samples.db")
c = small_db.cursor()

In [60]:
#query about postcodes.
small_postcode_query = "select tags.value, count(*) as count \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'postcode' \
        group by tags.value \
        order by count desc;"
c.execute(small_postcode_query)
small_postcodes = c.fetchall()
for row in small_postcodes:
    print(row)

('10314', 75)
('11234', 75)
('10312', 58)
('10306', 55)
('11706', 54)
('11743', 52)
('11236', 49)
('11385', 49)
('11223', 47)
('11757', 47)
('11207', 45)
('10469', 41)
('11746', 40)
('10465', 36)
('11208', 36)
('11357', 36)
('10304', 35)
('10305', 35)
('11219', 35)
('11203', 34)
('11229', 34)
('11413', 34)
('11704', 34)
('11204', 33)
('11220', 33)
('11434', 33)
('11731', 33)
('11221', 32)
('11420', 32)
('11214', 31)
('11364', 31)
('11375', 31)
('10461', 30)
('11412', 30)
('10309', 29)
('10466', 29)
('11210', 29)
('11233', 29)
('11358', 29)
('11215', 28)
('11228', 28)
('11235', 28)
('11373', 28)
('11378', 28)
('11379', 28)
('11725', 28)
('10308', 27)
('11209', 27)
('11218', 27)
('11377', 27)
('11717', 27)
('11729', 27)
('11365', 26)
('11419', 26)
('10301', 25)
('11230', 25)
('11355', 25)
('11701', 24)
('11361', 23)
('11421', 23)
('11435', 23)
('11768', 23)
('11795', 23)
('10310', 22)
('11216', 22)
('11367', 22)
('11368', 22)
('11432', 22)
('11691', 22)
('11702', 22)
('11105', 21)
('1142

In [61]:
#query about postcodes.
small_uncleaned_postcode_query = "select tags.value, count(*) as count \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'postcode' \
        group by tags.value \
        order by count desc;"
c.execute(small_uncleaned_postcode_query)
small_uncleaned_postcodes = c.fetchall()
for row in small_uncleaned_postcodes:
    print(row)

('10314', 75)
('11234', 75)
('10312', 58)
('10306', 55)
('11706', 54)
('11743', 52)
('11236', 49)
('11385', 49)
('11223', 47)
('11757', 47)
('11207', 45)
('10469', 41)
('11746', 40)
('10465', 36)
('11208', 36)
('11357', 36)
('10304', 35)
('10305', 35)
('11219', 35)
('11203', 34)
('11229', 34)
('11413', 34)
('11704', 34)
('11204', 33)
('11220', 33)
('11434', 33)
('11731', 33)
('11221', 32)
('11420', 32)
('11214', 31)
('11364', 31)
('11375', 31)
('10461', 30)
('11412', 30)
('10309', 29)
('10466', 29)
('11210', 29)
('11233', 29)
('11358', 29)
('11215', 28)
('11228', 28)
('11235', 28)
('11373', 28)
('11378', 28)
('11379', 28)
('11725', 28)
('10308', 27)
('11209', 27)
('11218', 27)
('11377', 27)
('11717', 27)
('11729', 27)
('11365', 26)
('11419', 26)
('10301', 25)
('11230', 25)
('11355', 25)
('11701', 24)
('11361', 23)
('11421', 23)
('11435', 23)
('11768', 23)
('11795', 23)
('10310', 22)
('11216', 22)
('11367', 22)
('11368', 22)
('11432', 22)
('11691', 22)
('11702', 22)
('11105', 21)
('1142

In [62]:
#query about websites.
small_site_query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'website' \
        order by tags.value;"
c.execute(small_site_query)
small_sites = c.fetchall()
for row in small_sites:
    print(row)

('http://brookwoodatbayshore.com',)
('http://citibikenyc.com',)
('http://davoodzadehrugs.com',)
('http://location.ctownsupermarkets.com/ny/jamaica/41-003/',)
('http://maps.google.com/local_url?dq=mi+Ranchito+Restaurant&q=http://www.miranchitogrillandbar.com/&oi=miw&sa=X&ct=miw_link&cd=1&cad=homepage,cid:1706131537235801532&ei=54vDUaLkB6fRxAGmtYDgBA&s=ANYYN7kQV_99A5w0y593WxyjRp5MfpJp2Q',)
('http://web.mta.info/nyct/service/bus/qnsche.htm',)
('http://www.dallasbbq.com',)
('http://www.dreamhotels.com/hotels/new-york/new-york/dream-downtown/hotel-overview',)
('http://www.dustyrosevintage.com',)
('http://www.jjay.cuny.edu/',)
('http://www.marrakechhotelnyc.com/',)
('http://www.nellyspillanes.com',)
('http://www.nyctacos.com/',)
('http://www.stagneschurchnyc.org',)
('http://www.thebeekmanpub.com/',)
('http://www.themexsoft.com',)
('http://www.whitecastle.com/locations/358',)
('https://www.shakeshack.com',)


In [63]:
#query about websites.
small_uncleaned_site_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'website' \
        order by tags.value;"
c.execute(small_uncleaned_site_query)
small_uncleaned_sites = c.fetchall()
for row in small_uncleaned_sites:
    print(row)

('brookwoodatbayshore.com',)
('http://citibikenyc.com',)
('http://davoodzadehrugs.com',)
('http://location.ctownsupermarkets.com/ny/jamaica/41-003/',)
('http://maps.google.com/local_url?dq=mi+Ranchito+Restaurant&q=http://www.miranchitogrillandbar.com/&oi=miw&sa=X&ct=miw_link&cd=1&cad=homepage,cid:1706131537235801532&ei=54vDUaLkB6fRxAGmtYDgBA&s=ANYYN7kQV_99A5w0y593WxyjRp5MfpJp2Q',)
('http://web.mta.info/nyct/service/bus/qnsche.htm',)
('http://www.dreamhotels.com/hotels/new-york/new-york/dream-downtown/hotel-overview',)
('http://www.dustyrosevintage.com',)
('http://www.jjay.cuny.edu/',)
('http://www.marrakechhotelnyc.com/',)
('http://www.nellyspillanes.com',)
('http://www.nyctacos.com/',)
('http://www.stagneschurchnyc.org',)
('http://www.thebeekmanpub.com/',)
('http://www.themexsoft.com',)
('http://www.whitecastle.com/locations/358',)
('https://www.shakeshack.com',)
('www.dallasbbq.com',)


In [64]:
#query about phones.
small_phone_query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'phone' \
        order by tags.value;"
c.execute(small_phone_query)
small_phones = c.fetchall()
for row in small_phones:
    print(row)

('(205) 900-7007',)
('(212) 222-2954',)
('(212) 268-5876',)
('(212) 462-0001',)
('(212) 749-1111',)
('(212) 792-9673',)
('(212) 874-4000',)
('(718) 482-7088',)
('(718) 657-0722',)


In [65]:
#query about phones.
small_uncleaned_phone_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'phone' \
        order by tags.value;"
c.execute(small_uncleaned_phone_query)
small_uncleaned_phones = c.fetchall()
for row in small_uncleaned_phones:
    print(row)

('+1 205 9007007',)
('+1 212 2222954',)
('+1 212 749 1111',)
('+1 212 874 4000',)
('+1-212-268-5876',)
('+1-212.462.0001',)
('+1-718-657-0722',)
('+12127929673',)
('1 718 482 7088',)


In [66]:
#query about faxs.
small_fax_query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'fax' \
        order by tags.value;"
c.execute(small_fax_query)
small_faxes = c.fetchall()
for row in small_faxes:
    print(row)

('(212) 268-0847',)
('(212) 678-6842',)


In [67]:
#query about faxs.
small_uncleaned_fax_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'fax' \
        order by tags.value;"
c.execute(small_uncleaned_fax_query)
small_uncleaned_faxes = c.fetchall()
for row in small_uncleaned_faxes:
    print(row)

('+1 212 678 6842',)
('+1-212-268-0847',)


In [68]:
#query about Faxs.
small_Fax_query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'Fax' \
        order by tags.value;"
c.execute(small_Fax_query)
small_Faxes = c.fetchall()
for row in small_Faxes:
    print(row)

In [69]:
#query about Faxs.
small_uncleaned_Fax_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'Fax' \
        order by tags.value;"
c.execute(small_uncleaned_Fax_query)
small_uncleaned_Faxes = c.fetchall()
for row in small_uncleaned_Faxes:
    print(row)

In [70]:
#query about maxspeed.
small_maxspeed_query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'maxspeed' \
        order by tags.value;"
c.execute(small_maxspeed_query)
small_maxspeeds = c.fetchall()
for row in small_maxspeeds:
    print(row)

('15 mph',)
('160 mph',)
('20 mph',)
('201 mph',)
('25 mph',)
('30 mph',)
('35 mph',)
('40 mph',)
('45 mph',)
('50 mph',)
('55 mph',)
('65 mph',)
('70 mph',)


In [71]:
#query about maxspeed.
small_uncleaned_maxspeed_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'maxspeed' \
        order by tags.value;"
c.execute(small_uncleaned_maxspeed_query)
small_uncleaned_maxspeeds = c.fetchall()
for row in small_uncleaned_maxspeeds:
    print(row)

('15',)
('15 mph',)
('160',)
('20 mph',)
('201',)
('25 mph',)
('30 mph',)
('35 mph',)
('40 mph',)
('45 mph',)
('50 mph',)
('55 mph',)
('65 mph',)
('70 mph',)
('Amtrak ETT 2010-01-18',)
('signals',)


In [72]:
small_db.close()

### - Found that the cleaning code is now without problem. Now I can apply the cleaning code to New_York_sample file.

# 3. Audit the cleaned data of osm_samples.db.

In [73]:
#Connect to osm_samples.db using sqlite3.
import sqlite3

db = sqlite3.connect("C:\\sqlite_windows\\osm_samples.db")
c = db.cursor()

In [74]:
#query about postcodes.
postcode_query = "select tags.value, count(*) as count \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'postcode' \
        group by tags.value \
        order by count desc;"
c.execute(postcode_query)
postcodes = c.fetchall()
for row in postcodes:
    print(row)

('10314', 769)
('11234', 678)
('10312', 594)
('10306', 534)
('11236', 510)
('11706', 507)
('11385', 502)
('11746', 477)
('11743', 468)
('11757', 438)
('11229', 411)
('11203', 398)
('10469', 388)
('11207', 369)
('11434', 366)
('10305', 362)
('11208', 360)
('11223', 360)
('11704', 355)
('11357', 346)
('11204', 345)
('11731', 342)
('10466', 328)
('11221', 325)
('11214', 324)
('11215', 323)
('10465', 319)
('11413', 319)
('11210', 315)
('11725', 315)
('11219', 314)
('10309', 311)
('11358', 307)
('11220', 306)
('11420', 306)
('10304', 302)
('11209', 300)
('11412', 300)
('11377', 298)
('11379', 296)
('11717', 292)
('10308', 290)
('11235', 284)
('10301', 283)
('10461', 283)
('11230', 281)
('11364', 280)
('11795', 280)
('11228', 279)
('11233', 274)
('11729', 271)
('11378', 267)
('11368', 262)
('11375', 261)
('11419', 260)
('11365', 250)
('11373', 250)
('11432', 250)
('11768', 248)
('11218', 246)
('11355', 241)
('11212', 230)
('11361', 230)
('11422', 227)
('11433', 223)
('11435', 221)
('11367', 

- All postcodes follow the 5 digit code or zip+4 code(xxxxx-xxxx) form.

In [75]:
#query about websites.
site_query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'website' \
        order by tags.value;"
c.execute(site_query)
sites = c.fetchall()
for row in sites:
    print(row)

('http://527madisonave.com/',)
('http://825thirdave.com/',)
('http://auttharosthainewyork.com',)
('http://baileyarboretum.org',)
('http://bathenhaven.com/',)
('http://bigtreebottles.com',)
('http://bk.com',)
('http://brooklynsandbox.com',)
('http://brooklynsocialbar.com/',)
('http://brookwoodatbayshore.com',)
('http://celticcorner.us/',)
('http://charno4.com/',)
('http://citibikenyc.com',)
('http://davoodzadehrugs.com',)
('http://doriangraynyc.com',)
('http://drambar.com/',)
('http://ezcesspoollongisland.com',)
('http://fkh.clarkschools.org/',)
('http://gomusicworks.com',)
('http://greenwichlocksmiths.com',)
('http://gsalehouse.com/',)
('http://hsbc.com',)
('http://jadisnyc.com/',)
('http://jerichoanimalhospital.com',)
('http://keyportlibrary.org/',)
('http://kfsmortgage.com',)
('http://landmarklivingston.com/',)
('http://learnandplayny.com',)
('http://lennysnyc.com/stores#lenny-s-55th-street',)
('http://lfstores.com/',)
('http://livinglotsnyc.org/lot/58276/',)
('http://location.ctowns

- All website addresses start with 'http' or 'https'.

In [76]:
#query about phones.
phone_query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'phone' \
        order by tags.value;"
c.execute(phone_query)
phones = c.fetchall()
for row in phones:
    print(row)

('(201) 224-8444',)
('(201) 330-1416',)
('(201) 398-0944',)
('(201) 444-8111',)
('(201) 455-9056',)
('(201) 612-0756',)
('(201) 865-3987',)
('(203) 324-1800',)
('(203) 531-7200',)
('(203) 703-9080',)
('(203) 863-3000',)
('(205) 900-7007',)
('(212) 222-2954',)
('(212) 224-4646',)
('(212) 227-5975',)
('(212) 228-5483',)
('(212) 229-1600',)
('(212) 238-7897',)
('(212) 242-6200',)
('(212) 254-0006',)
('(212) 254-1675',)
('(212) 260-7289',)
('(212) 260-8015',)
('(212) 265-7250',)
('(212) 268-5876',)
('(212) 280-8779',)
('(212) 340-0863',)
('(212) 353-3780',)
('(212) 462-0001',)
('(212) 502-4609',)
('(212) 517-6590',)
('(212) 663-1580',)
('(212) 663-6004',)
('(212) 665-9800',)
('(212) 675-0342',)
('(212) 675-0550',)
('(212) 686-6525',)
('(212) 687-1786',)
('(212) 690-4000',)
('(212) 691-6067',)
('(212) 749-1111',)
('(212) 792-9673',)
('(212) 799-9010',)
('(212) 874-4000',)
('(212) 912-9770',)
('(212) 966-0290',)
('(212) 979-0900',)
('(212) 982-0362',)
('(212) 982-2121',)
('(212) 983-5389',)


In [77]:
#query about faxs.
fax_query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'fax' \
        order by tags.value;"
c.execute(fax_query)
faxes = c.fetchall()
for row in faxes:
    print(row)

('(212) 242-1278',)
('(212) 268-0847',)
('(212) 678-6842',)
('(718) 268-7382',)
('(718) 361-2734',)
('(732) 543-2409',)
('(732) 548-8887',)
('(973) 761-9193',)


In [78]:
#query about Faxs.
Fax_query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'Fax' \
        order by tags.value;"
c.execute(Fax_query)
Faxes = c.fetchall()
for row in Faxes:
    print(row)

- All phone numbers and fax numbers follows the (xxx) xxx-xxxx format. And the key 'Fax' was changed to 'fax'.

In [79]:
#query about maxspeed.
maxspeed_query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'maxspeed' \
        order by tags.value;"
c.execute(maxspeed_query)
maxspeeds = c.fetchall()
for row in maxspeeds:
    print(row)

('10 mph',)
('110 mph',)
('125 mph',)
('15 mph',)
('160 mph',)
('177 mph',)
('20 mph',)
('201 mph',)
('217 mph',)
('25 mph',)
('30 mph',)
('35 mph',)
('40 mph',)
('45 mph',)
('48 mph',)
('50 mph',)
('55 mph',)
('60 mph',)
('64 mph',)
('65 mph',)
('70 mph',)
('72 mph',)
('75 mph',)
('80 mph',)
('90 mph',)


- All maxspeed values are numbers added by 'mph' unit. There isn't any more unrelated value in here.

# 4. Data exploration.

In [80]:
#Connect to osm_samples.db using sqlite3.
import sqlite3

db = sqlite3.connect("C:\\sqlite_windows\\osm_samples.db")
c = db.cursor()

In [87]:
#query about id's uniqueness.
is_id_unique_query = "select * \
        from ways join nodes on ways.id = nodes.id;"
#        group by nodes_tags.value \
#        order by num desc \
#        limit 6;"
c.execute(is_id_unique_query)
is_id_unique = c.fetchall()
for row in is_id_unique:
    print(row)

(477416562, 'Prof Eric Cameron', 5177123, '1', 46432976, '2017-02-27T05:51:25Z', 477416562, 40.9051277, -74.2250443, 'IanH', 306962, 2, 35003054, '2015-10-31T23:58:23Z')


In [88]:
id477416562_ways_query = "select * \
        from ways_tags where id = 477416562;"
#        group by nodes_tags.value \
#        order by num desc \
#        limit 6;"
c.execute(id477416562_ways_query)
id477416562_ways = c.fetchall()
for row in id477416562_ways:
    print(row)

(477416562, 'name', 'Morris Avenue', 'regular')
(477416562, 'highway', 'residential', 'regular')
(477416562, 'cfcc', 'A41', 'tiger')
(477416562, 'county', 'Bergen, NJ', 'tiger')
(477416562, 'reviewed', 'no', 'tiger')
(477416562, 'zip_left', '07026', 'tiger')
(477416562, 'name_base', 'Morris', 'tiger')
(477416562, 'name_type', 'Ave', 'tiger')
(477416562, 'zip_right', '07026', 'tiger')


In [89]:
id477416562_nodes_query = "select * \
        from nodes_tags where id = 477416562;"
#        group by nodes_tags.value \
#        order by num desc \
#        limit 6;"
c.execute(id477416562_nodes_query)
id477416562_nodes = c.fetchall()
for row in id477416562_nodes:
    print(row)

In [91]:
#query about id's dividness.
is_id_divide_query = "select * \
        from ways_tags join ways_nodes on ways_tags.id = ways_nodes.id join nodes_tags on ways_nodes.node_id = nodes_tags.id;"
#        group by nodes_tags.value \
#        order by num desc \
#        limit 6;"
c.execute(is_id_divide_query)
is_id_divide = c.fetchall()
for row in is_id_divide:
    print(row)

(5672946, 'name', 'West 108th Street', 'regular', 5672946, 595314104, 3, 595314104, 'highway', 'traffic_signals', 'regular')
(5672946, 'oneway', 'yes', 'regular', 5672946, 595314104, 3, 595314104, 'highway', 'traffic_signals', 'regular')
(5672946, 'highway', 'residential', 'regular', 5672946, 595314104, 3, 595314104, 'highway', 'traffic_signals', 'regular')
(5672946, 'cfcc', 'A41', 'tiger', 5672946, 595314104, 3, 595314104, 'highway', 'traffic_signals', 'regular')
(5672946, 'county', 'New York, NY', 'tiger', 5672946, 595314104, 3, 595314104, 'highway', 'traffic_signals', 'regular')
(5672946, 'reviewed', 'no', 'tiger', 5672946, 595314104, 3, 595314104, 'highway', 'traffic_signals', 'regular')
(5672946, 'zip_left', '10025', 'tiger', 5672946, 595314104, 3, 595314104, 'highway', 'traffic_signals', 'regular')
(5672946, 'name_base', '108th', 'tiger', 5672946, 595314104, 3, 595314104, 'highway', 'traffic_signals', 'regular')
(5672946, 'name_type', 'St', 'tiger', 5672946, 595314104, 3, 5953141

In [95]:
maxspeed_detail_query = "select distinct * \
        from (select * from uncleaned_nodes_tags union all select * from uncleaned_ways_tags) tags \
        where key = 'maxspeed';"
#        group by nodes_tags.value \
#        order by num desc \
#        limit 6;"
c.execute(maxspeed_detail_query)
maxspeed_detail_nodes = c.fetchall()
for row in maxspeed_detail_nodes:
    print(row)

(4412589465, 'maxspeed', '25 mph', 'regular')
(4412656420, 'maxspeed', '25 mph', 'regular')
(5668989, 'maxspeed', '25 mph', 'regular')
(5669809, 'maxspeed', '25 mph', 'regular')
(5670070, 'maxspeed', '25 mph', 'regular')
(5670172, 'maxspeed', '25 mph', 'regular')
(5670719, 'maxspeed', '25 mph', 'regular')
(5670792, 'maxspeed', '25 mph', 'regular')
(5670883, 'maxspeed', '25 mph', 'regular')
(5671317, 'maxspeed', '25 mph', 'regular')
(5671402, 'maxspeed', '25 mph', 'regular')
(5671688, 'maxspeed', '25 mph', 'regular')
(5672001, 'maxspeed', '25 mph', 'regular')
(5672273, 'maxspeed', '25 mph', 'regular')
(5672415, 'maxspeed', '25 mph', 'regular')
(5673526, 'maxspeed', '25 mph', 'regular')
(5711170, 'maxspeed', '25 mph', 'regular')
(5712041, 'maxspeed', '15 mph', 'regular')
(5714943, 'maxspeed', '25 mph', 'regular')
(5715001, 'maxspeed', '25 mph', 'regular')
(5715487, 'maxspeed', '15 mph', 'regular')
(5715934, 'maxspeed', '20 mph', 'regular')
(5716858, 'maxspeed', '25 mph', 'regular')
(5717

In [104]:
id11632247_nodes_query = "select * \
        from (select * from uncleaned_nodes_tags union all select * from uncleaned_ways_tags) tags \
        where id = 487847691;"
#        group by nodes_tags.value \
#        order by num desc \
#        limit 6;"
c.execute(id11632247_nodes_query)
id11632247_nodes = c.fetchall()
for row in id11632247_nodes:
    print(row)

(487847691, 'name', 'Northeast Corridor', 'regular')
(487847691, 'gauge', '1435', 'regular')
(487847691, 'usage', 'main', 'regular')
(487847691, 'railway', 'rail', 'regular')
(487847691, 'voltage', '12000', 'regular')
(487847691, 'maxspeed', '55 mph', 'regular')
(487847691, 'operator', 'Amtrak', 'regular')
(487847691, 'frequency', '25', 'regular')
(487847691, 'designation', '1', 'regular')
(487847691, 'electrified', 'contact_line', 'regular')
(487847691, 'maxspeed', 'Amtrak ETT 2010-01-18', 'source')
(487847691, 'old_railway_operator', 'PRR', 'regular')


### Top 5 usage of buildings.

In [113]:
#query about building.
building_query = "select tags.value, count(*) as num\
        from (select * from nodes_tags union all select * from ways_tags) tags\
        where tags.key = 'building' \
        group by tags.value \
        order by num desc \
        limit 6;"
c.execute(building_query)
buildings = c.fetchall()
for row in buildings:
    print(row)

('yes', 37626)
('house', 5393)
('garage', 4088)
('shed', 489)
('commercial', 317)
('service', 218)


### Top 5 sports facilities.

In [82]:
#query about shop.
shop_query = "select nodes_tags.value, count(*) as num\
        from nodes_tags \
            join (select distinct(id) from nodes_tags \
                where value = 'building') i\
            on nodes_tags.id = i.id \
        where nodes_tags.key = 'shop' \
        group by nodes_tags.value \
        order by num desc \
        limit 10;"
c.execute(shop_query)
shops = c.fetchall()
for row in shops:
    print(row)

### Top 5 usage of land.

In [83]:
#query about shop.
shop_query = "select nodes_tags.value, count(*) as num\
        from nodes_tags \
        where nodes_tags.key = 'shop' \
        group by nodes_tags.value \
        order by num desc \
        limit 5;"
c.execute(shop_query)
shops = c.fetchall()
for row in shops:
    print(row)

('clothes', 24)
('supermarket', 20)
('convenience', 9)
('beauty', 8)
('deli', 6)


In [84]:
#query about use.
use_query = "select tags.value, count(*) as num\
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'use' \
        group by tags.value \
        order by num desc \
        limit 11;"
c.execute(use_query)
uses = c.fetchall()
for row in uses:
    print(row)

('office', 1)
('shop', 1)


In [85]:
#query about usage.
usage_query = "select tags.value, count(*) as num\
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'usage' \
        group by tags.value \
        order by num desc \
        limit 11;"
c.execute(usage_query)
usages = c.fetchall()
for row in usages:
    print(row)

('main', 124)
('industrial', 40)
('branch', 28)
('military', 4)


In [86]:
#query about cusine.
cusine_query = "select tags.value, count(*) as num\
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'cusine' \
        group by tags.value \
        order by num desc \
        limit 11;"
c.execute(cusine_query)
cusines = c.fetchall()
for row in cusines:
    print(row)

('thai', 1)
